In [1]:
import pickle

with open('./datasets/small/HAS_e.pickle', 'rb') as f:
    HAS_e = pickle.load(f)

In [3]:
from utils import load_ldas, load_bertopics

ldas = load_ldas()
bertopics = load_bertopics()

In [24]:
from time import time
from utils import get_topics_lda, get_topics_bertopic, get_coherence, get_diversity

def evaluate_models(models, HAS_e, lda=False):
    r = [] # result
    for v in models:
        if lda:
            topics = get_topics_lda(models[v]['model'], HAS_e[v]['id2word'])
        else:
            topics = get_topics_bertopic(bertopics[v]['model'])
        t_c = time()
        c = get_coherence(
            topics=topics,
            texts=HAS_e[v]['T'],
            dictionary=HAS_e[v]['id2word']
        )
        t_d = time()
        d = get_diversity(topics)
        t_end = time()
        r.append({
            'variant': v,
            'training_time': models[v]['time'],
            'coherence': c,
            'coherence_time': t_d - t_c,
            'diversity': d,
            'diversity_time': t_end - t_d,
            'score': (c + d) / 2
        })
    return r

In [25]:
import pandas as pd

evaluation_ldas = pd.DataFrame(evaluate_models(ldas, HAS_e, lda=True))
evaluation_bertopics = pd.DataFrame(evaluate_models(bertopics, HAS_e))

In [26]:
evaluation_ldas

,variant,training_time,coherence,coherence_time,diversity,diversity_time,score
0,HT,19.907989,0.467677,2.772933,0.570370,0.000999,0.519024
1,HCLW,21.408583,0.385583,2.126000,0.839394,0.000000,0.612489
2,AT,29.401772,0.331885,4.615993,0.107813,0.000000,0.219849
3,ACLW,25.473022,0.322817,4.702602,0.285897,0.000000,0.304357


In [27]:
evaluation_bertopics

,variant,training_time,coherence,coherence_time,diversity,diversity_time,score
0,HT,19.220337,0.221388,1.701313,0.900000,0.0,0.560694
1,HCLW,6.492343,0.394938,1.601997,1.000000,0.0,0.697469
2,AT,25.480397,0.332414,3.849319,0.666667,0.0,0.499540
3,ACLW,29.705862,0.401106,3.482531,1.000000,0.0,0.700553
